In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import bloscpack as bp

import numpy as np

In [ ]:
features_dir = '/home/yuncong/csd395/CSHL_patch_features'

In [ ]:
train_features_dir = '/home/yuncong/csd395/CSHL_patch_features/train'

In [ ]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

In [ ]:
stack = 'MD589'

In [ ]:
features_allClasses_pos = {}
features_allClasses_neg = {}

In [ ]:
for label in labels:
    if label == 'BackG':
        continue
    features_allClasses_pos[label] = bp.unpack_ndarray_file(train_features_dir + '/%(stack)s_%(label)s_features.bp' % {'stack': stack, 'label': label})
    features_allClasses_neg[label] = bp.unpack_ndarray_file(train_features_dir + '/%(stack)s_%(label)s_surround_features.bp' % {'stack': stack, 'label': label})

In [ ]:
from sklearn.svm import SVC
from sklearn.externals import joblib

In [ ]:
stack = 'MD589'

In [ ]:
os.makedirs('svm')

In [ ]:
for label in labels:
    
    if label == 'BackG':
        continue
        
    print label
    
    features_allClasses_pos = bp.unpack_ndarray_file(train_features_dir + '/%(stack)s_%(label)s_features.bp' % {'stack': stack, 'label': label})
    features_allClasses_neg = bp.unpack_ndarray_file(train_features_dir + '/%(stack)s_%(label)s_surround_features.bp' % {'stack': stack, 'label': label})
    
    train_data = np.r_[features_allClasses_pos, features_allClasses_neg]
    train_labels = np.r_[np.ones((features_allClasses_pos.shape[0], )), 
                         -np.ones((features_allClasses_neg.shape[0], ))]
    
    svc = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, 
          probability=True, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
          max_iter=-1, decision_function_shape=None, random_state=None)
    
    svc.fit(train_data, train_labels)
    
    joblib.dump(svc, 'svm/%(label)s_svm.pkl' % {'label': label})
    
    del features_allClasses_pos, features_allClasses_neg, train_data, train_labels, svc

In [ ]:
svc_allClasses = {}
for label_ind, label in enumerate(labels[1:]):
    svc_allClasses[label] = joblib.load('svm/%(label)s_svm.pkl' % {'label': label})

In [ ]:
predictions_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_patch_predictions_svm'
if not os.path.exists(predictions_rootdir):
    os.makedirs(predictions_rootdir)

In [ ]:
def f(stack, sec):
    
    test_features_dir = features_dir + '/%(stack)s' % {'stack': stack, 'sec': sec}

#     features_roi = bp.unpack_ndarray_file(test_features_dir + '/%(stack)s_%(sec)04d_features.bp' % {'stack': stack, 'sec': sec})
    features_roi = load_hdf(test_features_dir + '/%(stack)s_%(sec)04d_features.hdf' % {'stack': stack, 'sec': sec})

    n = features_roi.shape[0]
#     print n

#     probs = np.zeros((n, len(labels)-1))

#     for label_ind, label in enumerate(labels[1:]):

    label = '7N'
    svc = svc_allClasses[label]
#     probs[:, label_ind] = svc.predict_proba(features_roi)[:, svc.classes_.tolist().index(1.)]

    svc.predict_proba(features_roi)[:, svc.classes_.tolist().index(1.)]
    
#     predictions_dir = predictions_rootdir + '/%(stack)s/%(sec)04d' % {'stack': stack, 'sec': sec}
#     if not os.path.exists(predictions_dir):
#         os.makedirs(predictions_dir)
    
#     np.save(predictions_dir + '/%(stack)s_%(sec)04d_roi1_predictions.npy' % {'stack': stack, 'sec': sec}, probs)

In [ ]:
stack = 'MD585'
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

In [ ]:
t = time.time()
Parallel(n_jobs=16)(delayed(f)(stack=stack, sec=sec) for sec in range(first_detect_sec, first_detect_sec+16))
print time.time() - t

In [ ]:
stack = 'MD585'
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

# sec = 142
for sec in range(first_detect_sec, last_detect_sec+1):
    
# test_features_dir = features_dir + '/%(stack)s/%(sec)04d' % {'stack': stack, 'sec': sec}
    test_features_dir = features_dir + '/%(stack)s' % {'stack': stack, 'sec': sec}

#     features_roi = bp.unpack_ndarray_file(test_features_dir + '/%(stack)s_%(sec)04d_features.bp' % {'stack': stack, 'sec': sec})
    features_roi = load_hdf(test_features_dir + '/%(stack)s_%(sec)04d_features.hdf' % {'stack': stack, 'sec': sec})

    n = features_roi.shape[0]

    probs = np.zeros((n, len(labels)-1))

    for label_ind, label in enumerate(labels[1:]):

        print label

        svc = svc_allClasses[label]
        probs[:, label_ind] = svc.predict_proba(features_roi)[:, svc.classes_.tolist().index(1.)]

In [ ]:
stack = 'MD585'
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

In [ ]:
for sec in range(first_detect_sec, last_detect_sec+1):
    print sec
    
# test_features_dir = features_dir + '/%(stack)s/%(sec)04d' % {'stack': stack, 'sec': sec}
    test_features_dir = features_dir + '/%(stack)s' % {'stack': stack, 'sec': sec}

    features_roi = bp.unpack_ndarray_file(test_features_dir + '/%(stack)s_%(sec)04d_features.bp' % {'stack': stack, 'sec': sec})
    
    save_hdf(features_roi, test_features_dir + '/%(stack)s_%(sec)04d_features.hdf' % {'stack': stack, 'sec': sec})